# CSV・APIを用いて休日データを取得

[内閣府のサイト](https://www8.cao.go.jp/chosei/shukujitsu/gaiyou.html)と
[日本の祝日API](https://holidays-jp.github.io/)を利用して振替休日を含んだデータをCSVファイルとして取得する．あるいはデータをひとまとめにしてCSVとして保存する．二つのフォーマットが少し異なるのと，振替休日の表記が異なることに注意

結果として，適切なフォーマットにして保存するプログラムにする．

In [3]:
import requests
from pathlib import Path
import pandas as pd
import datetime
import time
import io
import jpholiday
import numpy as np

## 内閣府のサイトから休日データを取得する 

### データを取得してみる 

requestによってcsvをメモリ上に保存し，pandas.Dataframeで取得している．このcsvを実際に開いてみれば分かるが，日付フォーマットが特殊(ex. 2020/1/1)なのでパースする必要がある．

In [78]:
url = "https://www8.cao.go.jp/chosei/shukujitsu/syukujitsu.csv"
req = requests.get(url)

csv_bin = io.BytesIO(req.content)

In [79]:
my_parser = lambda date: datetime.datetime.strptime(date, "%Y/%m/%d")

In [80]:
holiday_df_csv = pd.read_csv(csv_bin,
                             names=["date", "holiday_name"],
                             index_col="date",
                             parse_dates=True,
                             date_parser=my_parser,
                             header=0,
                             encoding="shift-jis"
                            )
holiday_df_csv

,holiday_name
date,
1955-01-01,元日
1955-01-15,成人の日
1955-03-21,春分の日
1955-04-29,天皇誕生日
1955-05-03,憲法記念日
...,...
2022-09-19,敬老の日
2022-09-23,秋分の日
2022-10-10,スポーツの日


In [81]:
holiday_df_csv.index.date

array([datetime.date(1955, 1, 1), datetime.date(1955, 1, 15),
       datetime.date(1955, 3, 21), datetime.date(1955, 4, 29),
       datetime.date(1955, 5, 3), datetime.date(1955, 5, 5),
       datetime.date(1955, 9, 24), datetime.date(1955, 11, 3),
       datetime.date(1955, 11, 23), datetime.date(1956, 1, 1),
       datetime.date(1956, 1, 15), datetime.date(1956, 3, 21),
       datetime.date(1956, 4, 29), datetime.date(1956, 5, 3),
       datetime.date(1956, 5, 5), datetime.date(1956, 9, 23),
       datetime.date(1956, 11, 3), datetime.date(1956, 11, 23),
       datetime.date(1957, 1, 1), datetime.date(1957, 1, 15),
       datetime.date(1957, 3, 21), datetime.date(1957, 4, 29),
       datetime.date(1957, 5, 3), datetime.date(1957, 5, 5),
       datetime.date(1957, 9, 23), datetime.date(1957, 11, 3),
       datetime.date(1957, 11, 23), datetime.date(1958, 1, 1),
       datetime.date(1958, 1, 15), datetime.date(1958, 3, 21),
       datetime.date(1958, 4, 29), datetime.date(1958, 5, 3),


### データを取得して適切な場所に適切なフォーマットで保存する関数

In [91]:
def make_source_with_naikaku(source_path):
    """
    内閣府のサイトから得られるデータ(https://www8.cao.go.jp/chosei/shukujitsu/syukujitsu.csv)
    を適切なフォーマット(header, indexの無い日にち，祝日名)に変換して引数のソースディレクトリに保存する関数．
    振替休日は休日と表記される．1955年からのデータであることに注意
    """
    url = "https://www8.cao.go.jp/chosei/shukujitsu/syukujitsu.csv"
    req = requests.get(url)
    csv_bin = io.BytesIO(req.content)
    my_parser = lambda date: datetime.datetime.strptime(date, "%Y/%m/%d")
    holiday_df_csv = pd.read_csv(csv_bin,
                             names=["date", "holiday_name"],
                             index_col="date",
                             parse_dates=True,
                             date_parser=my_parser,
                             header=0,
                             encoding="shift-jis"
                            )
    
    holiday_df_csv.to_csv(source_path, header=False)

In [92]:
source_path = Path("../source/holiday_naikaku.csv")
make_source_with_naikaku(source_path)

## APIを試す 

### jsonデータを取得 

In [82]:
url = "https://holidays-jp.github.io/api/v1/date.json"
req = requests.get(url)

In [83]:
req.json()

{'2020-01-01': '元日',
 '2020-01-13': '成人の日',
 '2020-02-11': '建国記念の日',
 '2020-02-23': '天皇誕生日',
 '2020-02-24': '天皇誕生日 振替休日',
 '2020-03-20': '春分の日',
 '2020-04-29': '昭和の日',
 '2020-05-03': '憲法記念日',
 '2020-05-04': 'みどりの日',
 '2020-05-05': 'こどもの日',
 '2020-05-06': '憲法記念日 振替休日',
 '2020-07-23': '海の日',
 '2020-07-24': '体育の日',
 '2020-08-10': '山の日',
 '2020-09-21': '敬老の日',
 '2020-09-22': '秋分の日',
 '2020-11-03': '文化の日',
 '2020-11-23': '勤労感謝の日',
 '2021-01-01': '元日',
 '2021-01-11': '成人の日',
 '2021-02-11': '建国記念の日',
 '2021-02-23': '天皇誕生日',
 '2021-03-20': '春分の日',
 '2021-04-29': '昭和の日',
 '2021-05-03': '憲法記念日',
 '2021-05-04': 'みどりの日',
 '2021-05-05': 'こどもの日',
 '2021-07-22': '海の日',
 '2021-07-23': '体育の日',
 '2021-08-08': '山の日',
 '2021-08-09': '休日 山の日',
 '2021-09-20': '敬老の日',
 '2021-09-23': '秋分の日',
 '2021-11-03': '文化の日',
 '2021-11-23': '勤労感謝の日',
 '2022-01-01': '元日',
 '2022-01-10': '成人の日',
 '2022-02-11': '建国記念の日',
 '2022-02-23': '天皇誕生日',
 '2022-03-21': '春分の日',
 '2022-04-29': '昭和の日',
 '2022-05-03': '憲法記念日',
 '2022-05-

振替休日も取得できている．また，内閣府のデータと違い振替休日の振替元を表示している．さらにフォーマットはすでに読み込める形になっている．

### CSVデータとして保存 

In [84]:
url = "https://holidays-jp.github.io/api/v1/date.csv"
csv_name = Path(url).name

req = requests.get(url)
csv_binary = req.content

with open(Path(csv_name), "wb") as f:
    f.write(csv_binary)

### 2015年から現在年までの休日を取得しCSVファイルに保存 

実は，2015年からしか祝日を取得できない．

In [85]:
date_holidayname = {"date":[], "holiday_name":[]}

for year in range(2015, datetime.datetime.now().year+1):
    url = "https://holidays-jp.github.io/api/v1/{}/date.json".format(year)
    req = requests.get(url)
    date_dict = req.json()
    
    date_holidayname["date"].extend(list(date_dict.keys()))
    date_holidayname["holiday_name"].extend(list(date_dict.values()))
    time.sleep(0.1)  # スクレイピングでサーバーの負荷を軽減するため
    
holiday_df_api = pd.DataFrame(date_holidayname)

In [46]:
holiday_df

,date,holiday_name
0,2015-01-01,元日
1,2015-01-12,成人の日
2,2015-02-11,建国記念の日
3,2015-03-21,春分の日
4,2015-04-29,昭和の日
...,...,...
123,2021-08-09,休日 山の日
124,2021-09-20,敬老の日
125,2021-09-23,秋分の日
126,2021-11-03,文化の日


In [47]:
file_name = "japanese_holiday_withapi.csv"

holiday_df.to_csv(file_name, index=False, header=False)

開いてみる

In [88]:
holiday_df_api = pd.read_csv(Path(file_name),
                             names=["date", "holiday_name"],
                             index_col="date",
                             parse_dates=True,
                            )

holiday_df_api

,holiday_name
date,
2015-01-01,元日
2015-01-12,成人の日
2015-02-11,建国記念の日
2015-03-21,春分の日
2015-04-29,昭和の日
...,...
2021-08-09,休日 山の日
2021-09-20,敬老の日
2021-09-23,秋分の日


In [87]:
holiday_df_api.index.date

array([datetime.date(2015, 1, 1), datetime.date(2015, 1, 12),
       datetime.date(2015, 2, 11), datetime.date(2015, 3, 21),
       datetime.date(2015, 4, 29), datetime.date(2015, 5, 3),
       datetime.date(2015, 5, 4), datetime.date(2015, 5, 5),
       datetime.date(2015, 5, 6), datetime.date(2015, 7, 20),
       datetime.date(2015, 9, 21), datetime.date(2015, 9, 22),
       datetime.date(2015, 9, 23), datetime.date(2015, 10, 12),
       datetime.date(2015, 11, 3), datetime.date(2015, 11, 23),
       datetime.date(2015, 12, 23), datetime.date(2016, 1, 1),
       datetime.date(2016, 1, 11), datetime.date(2016, 2, 11),
       datetime.date(2016, 3, 20), datetime.date(2016, 3, 21),
       datetime.date(2016, 4, 29), datetime.date(2016, 5, 3),
       datetime.date(2016, 5, 4), datetime.date(2016, 5, 5),
       datetime.date(2016, 7, 18), datetime.date(2016, 8, 11),
       datetime.date(2016, 9, 19), datetime.date(2016, 9, 22),
       datetime.date(2016, 10, 10), datetime.date(2016, 11, 3

### データを取得して適切な場所に適切なフォーマットで保存する関数

In [95]:
def make_source_with_api(source_path):
    """
    日本の祝日API(https://holidays-jp.github.io/)を用いて取得したデータを
    適切なフォーマット(header, indexの無い日にち，祝日名)に変換して引数のソースディレクトリに保存する関数．
    振替休日は振替元も同時に表示される．2015年からの祝日であることに注意
    """
    date_holidayname = {"date":[], "holiday_name":[]}

    for year in range(2015, datetime.datetime.now().year+1):
        url = "https://holidays-jp.github.io/api/v1/{}/date.json".format(year)
        req = requests.get(url)
        date_dict = req.json()

        date_holidayname["date"].extend(list(date_dict.keys()))
        date_holidayname["holiday_name"].extend(list(date_dict.values()))
        time.sleep(0.1)  # スクレイピングでサーバーの負荷を軽減するため

    holiday_df_api = pd.DataFrame(date_holidayname)
    holiday_df_api.to_csv(source_path, index=False, header=False)

In [96]:
source_path = Path("../source/holiday_api.csv")
make_source_with_api(source_path)

## JPholidayを利用する

jpholidayはデータの取得に時間がかかるので，csvに保存するだけでも速度上の効果がある．

In [6]:
start_date = datetime.date(1955,1,1)
end_date = datetime.date(datetime.datetime.now().year+1,12,31)
holidays_array = np.array(jpholiday.between(start_date, end_date))
holidays_array

array([[datetime.date(1955, 1, 1), '元日'],
       [datetime.date(1955, 1, 15), '成人の日'],
       [datetime.date(1955, 3, 21), '春分の日'],
       ...,
       [datetime.date(2022, 10, 10), 'スポーツの日'],
       [datetime.date(2022, 11, 3), '文化の日'],
       [datetime.date(2022, 11, 23), '勤労感謝の日']], dtype=object)

In [12]:
holiday_df_jpholiday = pd.DataFrame(holidays_array[:,1,None], 
                          columns=["holiday_name"],
                          index=pd.DatetimeIndex(holidays_array[:,0])
                         )
holiday_df_jpholiday.index.name = "date"
holiday_df_jpholiday

,holiday_name
date,
1955-01-01,元日
1955-01-15,成人の日
1955-03-21,春分の日
1955-04-29,天皇誕生日
1955-05-03,憲法記念日
...,...
2022-09-19,敬老の日
2022-09-23,秋分の日
2022-10-10,スポーツの日


In [15]:
def make_source_with_jpholiday(source_path):
    """
    jpholiday(https://pypi.org/project/jpholiday/)を用いて取得したデータを
    適切なフォーマット(header, indexの無い日にち，祝日名)に変換して引数のソースディレクトリに保存する関数．
    振替休日は振替元も同時に表示される．1955年からの祝日であることに注意
    """ 
    start_date = datetime.date(1955,1,1)
    end_date = datetime.date(datetime.datetime.now().year+1,12,31)
    holidays_array = np.array(jpholiday.between(start_date, end_date))
    
    holiday_df_jpholiday = pd.DataFrame(holidays_array[:,1,None], 
                          columns=["holiday_name"],
                          index=pd.DatetimeIndex(holidays_array[:,0])
                         )
    holiday_df_jpholiday.index.name = "date"
    holiday_df_jpholiday.to_csv(source_path, header=False)

In [16]:
source_path = Path("../source/holiday_jpholiday.csv")
make_source_with_jpholiday(source_path)